# NeuroMet

Instructions:
* this notebook cannot be used for two different analysis at the same time. If you are unsure if someone is using it click on File / Make a copy and work with the copy.
* to run a cell press Umschalt (Shift) + Enter
* to correctly run the workflow _EVERY_ cell has to be ran
* When you run a cell, there is at first an [*] on the left, then a number (maybe an updated number) when the cell ran.
* normally the only thing to adjust in the settings is 1.6 Subject

## 0. External Modules

red warnings are normal in next cell

In [ ]:
import importlib
import workflows
importlib.reload(workflows)
from workflows.NeuroMet_dev import NeuroMet

Just some external modules required to run correctly the notebook - nothing to edit - just press **shift + enter**

In [ ]:
import os
import glob
from IPython.display import Image
%matplotlib inline

## 1. Settings

### 1.1 Directories and Folders

In [ ]:
## Paths
raw_data_dir = '/media/drive_s/AG/AG-Floeel-Imaging/00-Rohdaten/02_niftis/NeuroMet_TheresaKoebe/02_NeuroMet'
base_dir = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET'
temp_dir = '/home/WorkFlowTemp/NeuroMet'

#Working directory
w_dir = os.path.join(base_dir, 'Structural_analysis')

### 1.2 Matlab

The path where SPM is and the command to call matlab. *Everything should be fine as is*

In [ ]:
spm_path='/opt/spm12'
matlab_command = "matlab -nodesktop -nosplash"

### 1.3 FSL

Set format for FSL output files, i.e. NIFTI_GZ, NIFTI

References: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FslEnvironmentVariables

In [ ]:
fsl_file_format = 'NIFTI_GZ'

### 1.4 QDec

In [ ]:
qdec_subfolder = ''

### 1.5 Others

* **cores** : How many cores should be available. That can mean the number of subject analyzed in parallel for one-core steps or the core available for a multi-core step.
    * If no other analysis are running on the computer, chose: c-3po: 4 and doc2: 3
    * If other analysis are running, chose: c-3po: 4-(used cores by other analysis) or doc2: 3-(used cores by other analysis) 
* **overwrite**: If the "_copy from raw data_" step should overwrite the copied files
* **omp_nthreads**: For the freesurfer steps with multicore support, **normally 1 is a good choice**
    * 4 cores at c-3po: for 1 subject --> omp_nthreads = 5
    * 4 cores at c-3po: for 2 subjects --> omp_nthreads = 2
    * 4 cores at c-3po: for 3 subjects and more --> omp_nthreads = 1

In [ ]:
cores = 1
overwrite = False
omp_nthreads = 1

### 1.5 Subjects

**subject_list**:
* to analyze all subjects cut&paste below: `subject_list = [ x.split('/')[-1][8:11] for x in os.listdir(w_dir) if x.startswith('NeuroMet')]`
* to analyse only some subjects use the format: `subject_list = ['subject number', 'subject number' ... ]` i.e.: `subject_list = ['001']` or `subject_list = ['001', '003']` 

**qdec_subjects**:
* to analyze all subjects in folder: `qdec_subjects = []`
* to analyze the same subjects as in `subject_list`: `qdec_subjects = subject_list`
* to analyse only some subjects use the format: `qdec_subjects = ['subject number', 'subject number' ... ]` i.e.: `qdec_subjects = ['001']` or `qdec_subjects = ['001', '003']` 

In [ ]:
subject_list = ['090']
qdec_subjects = []

## 2. Create a NeuroMet Object

In [ ]:
neuromet_creator = NeuroMet(subject_list, temp_dir, w_dir, omp_nthreads, raw_data_dir, overwrite, spm_path, matlab_command, fsl_file_format, qdec_subjects, qdec_subfolder)

# Workflow - 1st part - SPM Segment and Mask generation

## 3. Copy from raw data

In [ ]:
neuromet_creator.copy_from_raw_data()

In [ ]:
neuromet = neuromet_creator.make_neuromet1_workflow()

In [ ]:
neuromet.write_graph(graph2use = 'colored', dotfilename = './neuromet')
Image(filename='neuromet.png')

## 4. Run Pipeline

In [ ]:
%%time
neuromet.run('MultiProc', plugin_args={'n_procs': cores})

## Open UNI and UNI_DEN brainmasks -> edit

In [ ]:
mask_subject = '003'
viewer = 'fslview' # Possibilities: fslview, fsleyes, default = fslview
#viewer = '/opt/FSLeyes/fsleyes'

In [ ]:
%%bash -s "$w_dir" "$mask_subject" "$viewer"
SPMdir=$1
subject_number=$2
viewer=$3
# Maske gleichzeitig mit Unihintergrund öffnen

echo "w_dir=$SPMdir"
echo "subject_number=$subject_number"
echo "viewer=$viewer"

uniden="UNI"

subject="NeuroMet${subject_number}"
sub_dir=$SPMdir/${subject}
echo $subject

uni=$sub_dir/m${subject}.UNI_mp2rage_orig_reoriented.nii 
uni_mask=$sub_dir/${subject}.UNI_brain_bin.nii.gz
den_mask=$sub_dir/${subject}.DEN_brain_bin.nii.gz
echo $uni
echo $uni_mask
echo $den_mask

# Falls vorhanden, öffnen SPMbrain_bin
if [ -e $sub_dir/${subject}.SPMbrain_bin.nii.gz ]; then
    spmbrainbin="$sub_dir/${subject}.SPMbrain_bin.nii.gz"
fi

if [ $viewer = 'fslview' ]; then
    spmbrainbin="$spmbrainbin -l Green"
    fslview $uni $uni_mask -l Red $den_mask -l Yellow $spmbrainbin
    # $SPMdir/$subject.$ro_suffix $SPMdir/$subject.$suffix -l Red
elif [ $viewer = 'fsleyes' ]; then
    spmbrainbin="$spmbrainbin -cm green"
    /opt/FSLeyes/fsleyes $uni $uni_mask -cm red $den_mask -cm yellow $spmbrainbin
else
    spmbrainbin="$spmbrainbin -l Green"
    fslview $uni $uni_mask -l Red $den_mask -l Yellow $spmbrainbin
    # $SPMdir/$subject.$ro_suffix $SPMdir/$subject.$suffix -l Red
fi

### Which one has to be used as mask?

In [ ]:
%%bash -s "$w_dir" "$mask_subject"

mask='DEN'

SPMdir=$1
subject_number=$2
subject="NeuroMet${subject_number}"
sub_dir=$SPMdir/${subject}

cp $sub_dir/${subject}.${mask}_brain_bin.nii.gz $sub_dir/${subject}.SPMbrain_bin.nii.gz

# Workflow 2. Part - Create combined image and Freesurfer

## 05. Print current Subjects 

In [ ]:
print(subject_list)

## 06.Edit Subjects list 

**Subject List**:
* to analyze all subjects cut&paste below: `subject_list = [ x.split('/')[-1][8:11] for x in uni ]` already existing analyses will _NOT_ be run again neither overwritten
* to analyse only some subjects cut&paste below something with the format: `subject_list = ['subject number', 'subject number' ... ]` i.e.: `subject_list = ['001']` or `subject_list = ['001', '003']
* leave the next cell commented to mantain the actual list

In [ ]:
# Uncomment here for all subjects:
#subject_list = [ x.split('/')[-1][8:11] for x in uni ]

# only some subjects? Then edit here:  
#subject_list = ['001', '002'] # format: ['subid1', 'subid2', ....]

## Check if all the brain masks are present and gzipped

In [ ]:
neuromet_creator.check_gz_masks(subject_list)


## SetUp Workflow
Just run this cell to setup the Workflow - nothing to edit

In [ ]:
neuromet_creator = NeuroMet(subject_list, temp_dir, w_dir, omp_nthreads, raw_data_dir, overwrite, spm_path, matlab_command, fsl_file_format, qdec_subjects, qdec_subfolder)
neuromet2 = neuromet_creator.make_neuromet2_workflow()

## Draw Workflow's Graph
Run this cell to draw the workflow's graph

In [ ]:
neuromet2.write_graph(graph2use = 'colored', dotfilename = './neuromet_2')
Image(filename='neuromet_2.png')

## Run Freesurfer Analysis
run the cell below to run the analysis

In [ ]:
%%time
neuromet2.run('MultiProc', plugin_args={'n_procs': cores})

## 07. Qdec

In [ ]:
s = neuromet_creator.show_qdec_stuff()
print(s)

In [ ]:
%%bash -s "$w_dir"
SUBJECTS_DIR=$1
cd $SUBJECTS_DIR
qdec&

## 08. Extract Hippocampal volumes

In [ ]:
%%bash -s "$w_dir" "$subject_list"
SUBJECTS_DIR=$1
echo $FREESURFER_HOME
echo $SUBJECTS_DIR
subject_list=$2
cmd="/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/bash-scripts/HippocampalVolumeExtraction"
cd $SUBJECTS_DIR
$cmd ${subject_list//[\',\[\]]}

In [ ]:
import pandas as pd
out_file = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/Structural_analysis'
df = pd.read_csv(out_file, sep = ' ')
writer = pd.ExcelWriter('HippocampusSubfieldVolume.xlsx')
df.to_excel(writer,'HippocampusSubfieldVolume')
writer.save()

In [ ]:
## Vergleich 2 Freesurfer Ordner

In [ ]:
b = [ i.split('/')[-1] for i in glob.glob('/media/drive_s/AG/AG-Floeel-Imaging/02-User/TheresaKoebe/NeuroMet_Project/Structural_analysis/NeuroMet027/NeuroMet027.freesurfer/*/*')] 

In [ ]:
a = [ i.split('/')[-1] for i in glob.glob('/media/drive_s/AG/AG-Floeel-Imaging/02-User/TheresaKoebe/NeuroMet_Project/Structural_analysis/NeuroMet026/NeuroMet026.freesurfer/*/*')] 

In [ ]:
a

In [ ]:
set(a) - set(b)

In [ ]:
c = [ i.split('/')[-1] for i in glob.glob('/media/drive_s/AG/AG-Floeel-Imaging/02-User/TheresaKoebe/NeuroMet_Project/Structural_analysis/NeuroMet028/NeuroMet028.freesurfer/*/*')] 

In [ ]:
set(a) - set(c)